In [ ]:
from fastai.vision import *
from fastai.metrics import *
import numpy as np
import os
import shutil
import cv2 as cv
import matplotlib.pyplot as plt
from pathlib import Path

Google images search (type:face)

"jisoo blackpink"
"jennie blackpink"
"Rosé blackpink"
"lisa blackpink"

Scroll down to load more images until "Show more results" button comes up.
Paste this into console:

urls = Array.from(document.querySelectorAll('.rg_di .rg_meta')).map(el=>JSON.parse(el.textContent).ou);
window.open('data:text/csv;charset=utf-8,' + escape(urls.join('\n')));

Save the URLs as a csv.

# Nevermind this is trash. Use DownAlbum.

https://www.pinterest.ca/lulamulala/blackpink-jisoo/social-media/
https://www.pinterest.ca/lulamulala/blackpink-jisoo/photoshoot/
https://www.pinterest.ca/lulamulala/blackpink-jennie/social-media/
https://www.pinterest.ca/lulamulala/blackpink-jennie/photoshoot/
https://www.pinterest.ca/lulamulala/blackpink-rose/social-media/
https://www.pinterest.ca/lulamulala/blackpink-rose/photoshoot/
https://www.pinterest.ca/lulamulala/blackpink-lisa/social-media/
https://www.pinterest.ca/lulamulala/blackpink-lisa/photoshoot/

In [ ]:
classes = ["jisoo", "jennie", "rose", "lisa"]
path = Path('data')

In [ ]:
img_path = (path/classes[0]).ls()[0]
img = cv.imread(img_path.as_posix())
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

In [ ]:
plt.imshow(gray, cmap="gray")

In [ ]:
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
faces = face_cascade.detectMultiScale(gray, 1.3, 5)

In [ ]:
faces

In [ ]:
for (x,y,w,h) in faces:
    cv.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
plt.imshow(img, cmap="gray")

In [ ]:
for c in classes[0:4]:
    dest = path/c
    dest.mkdir(parents=True, exist_ok=True)
#     csv_file = [path for path in Path("csvs").ls() if c in path.as_posix()][0]
#     fastai.vision.download_images(csv_file, dest)
    fastai.vision.verify_images(dest, delete=True, max_size=500)
#     fastai.vision.verify_images(dest, delete=True, max_size=500)

In [ ]:
transforms = get_transforms(
    do_flip = True, #D
    flip_vert = False, #D
    max_zoom = 1,
    max_rotate = 10, #D
    max_lighting = 0.1, #default 0.2
    max_warp = 0.1, #default 0.1
    p_affine = 0.75, #D
    p_lighting = 0.75) #D

In [ ]:
torch.cuda.set_device(2)

In [ ]:
np.random.seed(42)

data = (ImageDataBunch
        .from_folder(path,
                     train=".", valid_pct=0.2, bs=64,
                     ds_tfms=transforms, size=336)
#         .filter_by_func(lambda fname:"Blackpink" in Path(fname).as_posix())
        .normalize(imagenet_stats))

In [ ]:
data

In [ ]:
data.show_batch(rows=3, figsize=(7,8))

In [ ]:
learn = cnn_learner(data, models.resnet50, metrics=error_rate)

In [ ]:
lr_find(learn)
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(10)

In [ ]:
learn.recorder.plot_losses()

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
losses,idxs = interp.top_losses()

In [ ]:
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(20, figsize=(15,11))

In [ ]:
interp.most_confused(min_val=2)

In [ ]:
model_path = Path("..")
learn.save(model_path/"2019-06-23_RESNET50_defaults_10epoch_0.2error")

In [ ]:
!jupyter nbconvert celebrity-classifier --to html --output nbs/2019-06-25_RESNET50_defaults_0.8accuracy_LOL